# MA6202: Laboratorio de Ciencia de Datos

### **Profesor:** Nicolás Caro

### 12/07/2020 - Tarea 2


### **Integrantes del grupo**: Diego Irarrazaval, 

# Pregunta 1: Carga y transformacion de datos.

In [ ]:
#Estas lineas corren solo en google colab:
import os.path 
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  if os.path.exists('/content/ChestXRay2017.zip'):
    print("Datos ya descargados")
  else:
    !wget https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip
    !unzip /content/ChestXRay2017.zip

In [ ]:
import torch